In [2]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri = MLFLOW_TRACKING_URI)

# Interacting with the MLflow tracker server
### MlflowClient object allows us to interact with..
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions.

#### To instantiate it we need to pass a tracking URI and/or a registry URI

In [3]:
#list of experiments on our mlflow ui
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [4]:
#check for the given experiment, what are the best runs/models

from mlflow.entities import ViewType

runs  = client.search_runs(
    experiment_ids="1",
    filter_string ="metrics.rmse <6.8",
    run_view_type = ViewType.ACTIVE_ONLY,
    max_results = 5,
    order_by = ['metrics.rmse ASC']
)

In [5]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse:{run.data.metrics['rmse']:.4f}")

run id: b737d95fc8a14ae2a599771751638cb0, rmse:6.2887
run id: e389dae9d8a045538a7330c618cb7c96, rmse:6.2902
run id: 4baf368bdf8e4633bdd0d3e7dc2c3560, rmse:6.2913
run id: 7338b6a818bf4893b4ccc0887aab1148, rmse:6.3040
run id: 80fe68adcdce401699fd14e4a906c162, rmse:6.3093


# Interacting with the Model Registry
#### In this section We will use the MlflowClient instance to:

1. Register a new version for the experiment nyc-taxi-regressor
2. Retrieve the latests versions of the model nyc-taxi-regressor and check that a new version 4 was created.
3. Transition the version 4 to "Staging" and adding annotations to it.

In [14]:
#promote some of the above models to model registry
import mlflow

#need to set also here the tracking server otherwise it would try to look for the models in 
#the local folder
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [15]:
#register a new version of the model
run_id = '80fe68adcdce401699fd14e4a906c162'
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")


Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2022/07/28 16:16:36 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1659005196460, current_stage='None', description=None, last_updated_timestamp=1659005196460, name='nyc-taxi-regressor', run_id='80fe68adcdce401699fd14e4a906c162', run_link=None, source='./mlruns/1/80fe68adcdce401699fd14e4a906c162/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [17]:
client.list_registered_models()

[<RegisteredModel: creation_timestamp=1659001275504, description='The NYC Taxi predictor for trip duration', last_updated_timestamp=1659005196460, latest_versions=[<ModelVersion: creation_timestamp=1659001302451, current_stage='Staging', description='', last_updated_timestamp=1659001594486, name='nyc-taxi-regressor', run_id='08062686756249d8ba1a9b5920d2f1a2', run_link='', source='./mlruns/1/08062686756249d8ba1a9b5920d2f1a2/artifacts/model', status='READY', status_message=None, tags={'model': 'Gradient boosting regressor'}, user_id=None, version=2>,
  <ModelVersion: creation_timestamp=1659005196460, current_stage='None', description=None, last_updated_timestamp=1659005196460, name='nyc-taxi-regressor', run_id='80fe68adcdce401699fd14e4a906c162', run_link=None, source='./mlruns/1/80fe68adcdce401699fd14e4a906c162/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>], name='nyc-taxi-regressor', tags={}>]

In [6]:
#get latest version of the model

model_name='nyc-taxi-regressor'
latest_versions = client.get_latest_versions(name = model_name)

In [7]:
for version in latest_versions:
    print(f'version: {version.version}, stage: {version.current_stage}')

version: 3, stage: Staging


In [9]:
#transition a model from one stage to another
model_version =3
new_stage = 'Staging'

client.transition_model_version_stage(
    name = model_name,
    version = model_version,
    stage=new_stage,
    archive_existing_versions = False
)

In [10]:
#we can update a version also
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name= model_name,
    version=3,
    description = f'The model version {model_version} was transitioned to {new_stage} on {date}')

<ModelVersion: creation_timestamp=1659005196460, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2022-07-31', last_updated_timestamp=1659245830063, name='nyc-taxi-regressor', run_id='80fe68adcdce401699fd14e4a906c162', run_link=None, source='./mlruns/1/80fe68adcdce401699fd14e4a906c162/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

# Comparing versions and selecting the new "Production" model
In this last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the DictVectorizer that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the DictVectorizer so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.
#### Note: the model registry doesn't actually deploy the model to production when we transition a model to the "Production" stage, it just assign a label to that model version. We should complement the registry with some CI/CD code that does the actual deployment.

In [12]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    #fn to read the dataframe in .parquet file
    df = pd.read_csv(filename)

#following lines to change dropoff time from string to datetime object if dataframe is in .csv file
#     df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
#     df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
#preprocess the data using the preprocessor 'dv' that we have pickeled
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    '''
    Given a stage, test set and the name of the model and make prediction on the test set
    '''
    #load the model
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    
    #make prediction
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [21]:
#using the same run_id that we used to register a version
run_id = '25fea966d33c482082a2ec0334b85862'  #this run_id have the artifact file on the MLflow


#download the preprocessor that we saved in artifact
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/home/iiitd/Documents/MLOPs/01-experiment/preprocessor'